# Capstone Project - IMDB Data Set (Phase I and II)

### Submitted by: Jwala R (Batch DS0822)

Phase I - Web Scrapping
Phase II - Querying using SQLite and Pandas SQL

The dataset taken into consideration is IMDB dataset. For the purpose of study, I have the top taken 1000 records available in the URL https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1

# WEB SCRAPPING

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the number of movies to scrape
num_movies = 1000

# Define the URL pattern for pagination
url_pattern = "https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1&start={}"

# Define lists to store the extracted data
first_csv_data = []
second_csv_data = []

# Scrape data from multiple pages
for start in range(1, num_movies + 1, 50):
    # Construct the URL for each page
    url = url_pattern.format(start)
    
    # Send a GET request to the IMDb URL
    response = requests.get(url)
    
    # Create BeautifulSoup object and find all movie items
    soup = BeautifulSoup(response.content, "html.parser")
    movie_items = soup.find_all("div", class_="lister-item-content")
    
    # Extract data for the first CSV file
    for item in movie_items:
        sno = item.find("span", class_="lister-item-index").text.replace(".", "")
        title = item.find("h3", class_="lister-item-header").find("a").text
        directors = item.find("p", class_="").find_all("a")
        director_names = [director.text for director in directors]
        duration = item.find("span", class_="runtime").text.strip(" min")
        year = item.find("span", class_="lister-item-year").text.strip("()")
        rating_element = item.find("div", class_="inline-block ratings-imdb-rating")
        rating = rating_element.strong.text if rating_element else "N/A"
        metascore_element = item.find("span", class_="metascore")
        metascore = metascore_element.text.strip() if metascore_element else "N/A"
        
        first_csv_data.append([sno, title, ", ".join(director_names), duration, year, rating, metascore])
    
    # Extract data for the second CSV file
    for item in movie_items:
        title = item.find("h3", class_="lister-item-header").find("a").text
        stars = item.find("p", class_="").find_all("a")[1:5]  # Exclude the first link (director)
        star_names = [star.text for star in stars]
        votes_element = item.find("p", class_="sort-num_votes-visible").find_all("span")
        votes = votes_element[1].text.replace(",", "") if len(votes_element) >= 2 else "N/A"
        genre_element = item.find("span", class_="genre")
        genre = genre_element.text.strip().split(", ") if genre_element else []
        gross_element = item.find("p", class_="sort-num_votes-visible").find_all("span")
        gross = gross_element[-1]["data-value"] if gross_element else "N/A"
        certification_element = item.find("span", class_="certificate")
        certification = certification_element.text.strip() if certification_element else "N/A"
        
        second_csv_data.append([title, ", ".join(star_names), votes, ", ".join(genre), gross, certification])

# Create dataframes from the extracted data
first_csv_df = pd.DataFrame(first_csv_data, columns=["Sno", "Movie Name", "Director Name", "Duration", "Year", "Ratings", "Metascore"])
second_csv_df = pd.DataFrame(second_csv_data, columns=["Movie Name", "Stars", "Votes", "Genre", "Gross Collection", "Certification"])

# Save dataframes to CSV files
first_csv_df.to_csv("first_csv_data.csv", index=False)
second_csv_df.to_csv("second_csv_data.csv", index=False)


In [21]:
first_csv_df

,Sno,Movie Name,Director Name,Duration,Year,Ratings,Metascore
0,1,The Dark Knight,"Christopher Nolan, Christian Bale, Heath Ledge...",152,2008,9.0,84
1,2,The Lord of the Rings: The Return of the King,"Peter Jackson, Elijah Wood, Viggo Mortensen, I...",201,2003,9.0,94
2,3,Spider-Man: Across the Spider-Verse,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...",140,2023,8.9,86
3,4,Inception,"Christopher Nolan, Leonardo DiCaprio, Joseph G...",148,2010,8.8,74
4,5,The Lord of the Rings: The Fellowship of the Ring,"Peter Jackson, Elijah Wood, Ian McKellen, Orla...",178,2001,8.8,92
...,...,...,...,...,...,...,...
995,996,Baiohazâdo: Dijenerêshon,"Makoto Kamiya, Paul Mercier, Alyson Court, Lau...",97,2008,6.4,N/A
996,997,Dnevnoy dozor,"Timur Bekmambetov, Konstantin Khabenskiy, Mari...",132,2006,6.4,59
997,998,Redirected,"Emilis Velyvis, Vinnie Jones, Scot Williams, G...",99,2014,6.4,N/A
998,999,Transformers: Rise of the Beasts,"Steven Caple Jr., Anthony Ramos, Dominique Fis...",127,2023,6.3,42


In [22]:
second_csv_df

,Movie Name,Stars,Votes,Genre,Gross Collection,Certification
0,The Dark Knight,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",2738965,"Action, Crime, Drama","534,858,444",UA
1,The Lord of the Rings: The Return of the King,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",1897977,"Action, Adventure, Drama","377,845,905",U
2,Spider-Man: Across the Spider-Verse,"Kemp Powers, Justin K. Thompson, Shameik Moore...",182105,"Animation, Action, Adventure",182105,U
3,Inception,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",2430566,"Action, Adventure, Sci-Fi","292,576,195",UA
4,The Lord of the Rings: The Fellowship of the Ring,"Elijah Wood, Ian McKellen, Orlando Bloom, Sean...",1926413,"Action, Adventure, Drama","315,544,750",U
...,...,...,...,...,...,...
995,Baiohazâdo: Dijenerêshon,"Paul Mercier, Alyson Court, Laura Bailey, Roge...",32289,"Animation, Action, Horror",32289,R
996,Dnevnoy dozor,"Konstantin Khabenskiy, Mariya Poroshina, Vladi...",34964,"Action, Fantasy, Horror","450,686",R
997,Redirected,"Vinnie Jones, Scot Williams, Gil Darnell, Oliv...",34350,"Action, Comedy, Crime",34350,N/A
998,Transformers: Rise of the Beasts,"Anthony Ramos, Dominique Fishback, Luna Lauren...",44774,"Action, Adventure, Sci-Fi",44774,UA


# TABLE CREATION

In [42]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Create the first table
cursor.execute('''CREATE TABLE IF NOT EXISTS first_table (
                    Sno INTEGER,
                    Movie_Name TEXT,
                    Director_Name TEXT,
                    Duration REAL,
                    Year TEXT,
                    Ratings REAL,
                    Metascore REAL
                )''')

# Create the second table
cursor.execute('''CREATE TABLE IF NOT EXISTS second_table (
                    Movie_Name TEXT,
                    Stars TEXT,
                    Votes INTEGER,
                    Genre TEXT,
                    Gross_Collection REAL,
                    Certification TEXT
                )''')

# Read data from the first CSV file
first_csv_df = pd.read_csv('first_csv_data.csv')

# Insert data into the first table
for _, row in first_csv_df.iterrows():
    year = row['Year'].replace('(', '').replace(')', '')  # Remove parentheses from the value
    cursor.execute('''INSERT INTO first_table (Sno, Movie_Name, Director_Name, Duration, Year, Ratings, Metascore)
                      VALUES (?, ?, ?, ?, ?, ?, ?)''',
                   (row['Sno'], row['Movie Name'], row['Director Name'], float(row['Duration']), year,
                    float(row['Ratings']), float(row['Metascore'])))

# Read data from the second CSV file
second_csv_df = pd.read_csv('second_csv_data.csv')

# Insert data into the second table
for _, row in second_csv_df.iterrows():
    gross_collection = row['Gross Collection'].replace(',', '')  # Remove commas from the value
    cursor.execute('''INSERT INTO second_table (Movie_Name, Stars, Votes, Genre, Gross_Collection, Certification)
                      VALUES (?, ?, ?, ?, ?, ?)''',
                   (row['Movie Name'], row['Stars'], int(row['Votes']), row['Genre'],
                    float(gross_collection), row['Certification']))

# Commit the changes and close the connection
conn.commit()
conn.close()



# QUERY USING SQLITE

In [52]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# 1) Display all the details of movies created by directors Christopher and Matt Reeves using LIKE
cursor.execute('''SELECT *
                  FROM first_table
                  WHERE Director_Name LIKE '%Christopher%' OR Director_Name LIKE '%Matt Reeves%' ''')
result1 = cursor.fetchall()
print("Movies created by directors Christopher and Matt Reeves:")
for row in result1:
    print(row)
print()

# Close the connection
conn.close()


Movies created by directors Christopher and Matt Reeves:
(1, 'The Dark Knight', 'Christopher Nolan, Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine', 152.0, 2008.0, 9.0, 84.0)
(4, 'Inception', 'Christopher Nolan, Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page, Ken Watanabe', 148.0, 2010.0, 8.8, 74.0)
(22, 'The Dark Knight Rises', 'Christopher Nolan, Christian Bale, Tom Hardy, Anne Hathaway, Gary Oldman', 164.0, 2012.0, 8.4, 78.0)
(40, 'Batman Begins', 'Christopher Nolan, Christian Bale, Michael Caine, Ken Watanabe, Liam Neeson', 140.0, 2005.0, 8.2, 70.0)
(57, 'Mission: Impossible - Dead Reckoning Part One', 'Christopher McQuarrie, Tom Cruise, Hayley Atwell, Ving Rhames, Simon Pegg', 163.0, 2023.0, 8.1, 81.0)
(67, 'How to Train Your Dragon', 'Dean DeBlois, Chris Sanders, Jay Baruchel, Gerard Butler, Christopher Mintz-Plasse, Craig Ferguson', 98.0, 2010.0, 8.1, 75.0)
(131, 'The Batman', 'Matt Reeves, Robert Pattinson, Zoë Kravitz, Jeffrey Wright, Colin Farrell', 176.0, 2

In [53]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# 2) Display all the details of movies with a duration of 140 minutes to 190 minutes
cursor.execute('''SELECT *
                  FROM first_table
                  WHERE Duration BETWEEN 140 AND 190''')
result2 = cursor.fetchall()

# Print the details of movies with a duration of 140 minutes to 190 minutes
print("Movies with a duration of 140 minutes to 190 minutes:")
for row in result2:
    print(row)
print()

# Close the connection
conn.close()


Movies with a duration of 140 minutes to 190 minutes:
(1, 'The Dark Knight', 'Christopher Nolan, Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine', 152.0, 2008.0, 9.0, 84.0)
(3, 'Spider-Man: Across the Spider-Verse', 'Joaquim Dos Santos, Kemp Powers, Justin K. Thompson, Shameik Moore, Hailee Steinfeld, Brian Tyree Henry, Luna Lauren Velez', 140.0, 2023.0, 8.9, 86.0)
(4, 'Inception', 'Christopher Nolan, Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page, Ken Watanabe', 148.0, 2010.0, 8.8, 74.0)
(5, 'The Lord of the Rings: The Fellowship of the Ring', 'Peter Jackson, Elijah Wood, Ian McKellen, Orlando Bloom, Sean Bean', 178.0, 2001.0, 8.8, 92.0)
(6, 'The Lord of the Rings: The Two Towers', 'Peter Jackson, Elijah Wood, Ian McKellen, Viggo Mortensen, Orlando Bloom', 179.0, 2002.0, 8.8, 87.0)
(9, 'Soorarai Pottru', 'Sudha Kongara, Suriya, Paresh Rawal, Aparna Balamurali, Urvashi', 153.0, 2020.0, 8.7, None)
(14, 'Sita Ramam', 'Hanu Raghavapudi, Dulquer Salmaan, Mrunal Thakur, Ras

In [54]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# 3) Display all details of movies with ratings above 7 in ascending order
cursor.execute('''SELECT *
                  FROM first_table
                  WHERE Ratings > 7
                  ORDER BY Ratings ASC''')
result3 = cursor.fetchall()

# Print the details of movies with ratings above 7 in ascending order
print("Movies with ratings above 7 in ascending order:")
for row in result3:
    print(row)
print()

# Close the connection
conn.close()


Movies with ratings above 7 in ascending order:
(432, 'The Flash', 'Andy Muschietti, Ezra Miller, Michael Keaton, Sasha Calle, Michael Shannon', 144.0, 2023.0, 7.1, 56.0)
(433, 'Extraction 2', 'Sam Hargrave, Chris Hemsworth, Golshifteh Farahani, Adam Bessa, Tornike Gogrichiani', 122.0, 2023.0, 7.1, 57.0)
(434, 'Mission: Impossible', 'Brian De Palma, Tom Cruise, Jon Voight, Emmanuelle Béart, Henry Czerny', 110.0, 1996.0, 7.1, 59.0)
(435, 'Man of Steel', 'Zack Snyder, Henry Cavill, Amy Adams, Michael Shannon, Diane Lane', 143.0, 2013.0, 7.1, 55.0)
(436, 'The Mummy', 'Stephen Sommers, Brendan Fraser, Rachel Weisz, John Hannah, Arnold Vosloo', 124.0, 1999.0, 7.1, 48.0)
(437, 'Tropic Thunder', 'Ben Stiller, Ben Stiller, Jack Black, Robert Downey Jr., Jeff Kahn', 107.0, 2008.0, 7.1, 71.0)
(438, 'Wrath of Man', 'Guy Ritchie, Jason Statham, Holt McCallany, Josh Hartnett, Rocci Boy Williams', 119.0, 2021.0, 7.1, 57.0)
(439, 'Free Guy', 'Shawn Levy, Ryan Reynolds, Jodie Comer, Taika Waititi, Lil

In [66]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# 4) Display all movie names in descending order without repetition
cursor.execute('''SELECT DISTINCT Movie_Name
                  FROM first_table
                  ORDER BY Movie_Name DESC''')
result4 = cursor.fetchall()

# Print the movie names in descending order without repetition
print("Movie names in descending order without repetition:")
for row in result4:
    print(row[0])
print()

# Close the connection
conn.close()



Movie names in descending order without repetition:
Ölümlü Dünya
Zombieland: Double Tap
Zombieland
Zatôichi
Zack Snyder's Justice League
Yôjinbô
Young Guns II
Young Guns
You Only Live Twice
Yip Man chin chyun
Yip Man 4
Yip Man 3
Yip Man 2
Ying xiong
Yi dai zong shi
X: First Class
X2
X-Men: The Last Stand
X-Men: Days of Future Past
X-Men: Apocalypse
X-Men Origins: Wolverine
X-Men
Wyatt Earp
Wrath of Man
World War Z
Wonder Woman
WolfWalkers
Wo hu cang long
Willow
Wild Target
White Heat
Where Eagles Dare
What Happened to Monday
Westworld
We Were Soldiers
We Own the Night
Way Down
Watchmen
Wasabi
Warrior
Warcraft
WarGames
War of the Worlds
War for the Planet of the Apes
War Horse
War
Wanted
Waar
Vishwaroopam
Violent Night
Vikram Vedha
Vikram
Venom
Vantage Point
Vanishing Point
Vampire Hunter D: Bloodlust
Valerian and the City of a Thousand Planets
VR (Vikrant Rona)
V for Vendetta
Uri: The Surgical Strike
Upgrade
Unstoppable
Unleashed
Unknown
United 93
Underworld: Rise of the Lycans
Underwo

In [67]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# 5) Display movie names that start with 'P' and their rating is greater than 7 without repetition
cursor.execute('''SELECT DISTINCT Movie_Name
                  FROM first_table
                  WHERE Movie_Name LIKE 'P%' AND Ratings > 7''')
result5 = cursor.fetchall()

# Print the movie names that start with 'P' and have a rating greater than 7 without repetition
print("Movie names starting with 'P' and rating greater than 7 without repetition:")
for row in result5:
    print(row[0])
print()

# Close the connection
conn.close()



Movie names starting with 'P' and rating greater than 7 without repetition:
Paan Singh Tomar
Pirates of the Caribbean: The Curse of the Black Pearl
Per un pugno di dollari
Predator
Ponniyin Selvan: I
Pushpa: The Rise - Part 1
Predestination
Pirates of the Caribbean: Dead Man's Chest
Patriots Day
Point Break
Prey
Pirates of the Caribbean: At World's End
Planet Terror
Payback



In [68]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# 1) Display all movie names with star Arnold Schwarzenegger in ascending order without repetition
cursor.execute('''SELECT DISTINCT Movie_Name
                  FROM second_table
                  WHERE Stars LIKE '%Arnold Schwarzenegger%'
                  ORDER BY Movie_Name ASC''')
result1 = cursor.fetchall()

# Print the movie names with the star Arnold Schwarzenegger in ascending order without repetition
print("Movie names with star Arnold Schwarzenegger in ascending order without repetition:")
for row in result1:
    print(row[0])
print()

# Close the connection
conn.close()



Movie names with star Arnold Schwarzenegger in ascending order without repetition:
Commando
Conan the Barbarian
Escape Plan
Last Action Hero
Predator
Terminator 2: Judgment Day
The Running Man
The Terminator
Total Recall
True Lies



In [64]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# 2) Display all details of the movie with the highest number of votes without repetition
cursor.execute('''SELECT *
                  FROM second_table
                  WHERE Votes = (SELECT MAX(Votes) FROM second_table)
                  LIMIT 1''')
result2 = cursor.fetchall()

# Print the details of the movie with the highest number of votes
print("Details of the movie with the highest number of votes:")
for row in result2:
    print(row)
print()

# Close the connection
conn.close()


Details of the movie with the highest number of votes:
('The Dark Knight', 'Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine', 2738965, 'Action, Crime, Drama', 534858444.0, 'UA')



In [62]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# 3) Display movie names with gross collections in descending order without repetition
cursor.execute('''SELECT DISTINCT Movie_Name
                  FROM second_table
                  ORDER BY Gross_Collection DESC''')
result3 = cursor.fetchall()

# Print the movie names with gross collections in descending order without repetition
print("Movie names with gross collections in descending order without repetition:")
for row in result3:
    print(row[0])
print()

# Close the connection
conn.close()


Movie names with gross collections in descending order without repetition:
Star Wars: Episode VII - The Force Awakens
Avengers: Endgame
Spider-Man: No Way Home
Avatar
Top Gun: Maverick
Black Panther
Avengers: Infinity War
Avatar: The Way of Water
Jurassic World
The Avengers
Star Wars: Episode VIII - The Last Jedi
Incredibles 2
The Dark Knight
Rogue One
Star Wars: Episode IX - The Rise of Skywalker
Star Wars: Episode I - The Phantom Menace
Avengers: Age of Ultron
Black Panther: Wakanda Forever
The Dark Knight Rises
Captain Marvel
The Hunger Games: Catching Fire
Pirates of the Caribbean: Dead Man's Chest
Wonder Woman
Doctor Strange in the Multiverse of Madness
Iron Man Three
Captain America: Civil War
The Hunger Games
Jumanji: Welcome to the Jungle
Spider-Man
Jurassic Park
Spider-Man: Far from Home
Guardians of the Galaxy Vol. 2
Star Wars: Episode III - Revenge of the Sith
The Lord of the Rings: The Return of the King
Spider-Man 2
The Batman
Deadpool
Fast & Furious 7
American Sniper
The 

In [69]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# 4) Display the gross collection of movies with the star Arnold without repetition
cursor.execute('''SELECT DISTINCT Movie_Name, Gross_Collection
                  FROM second_table
                  WHERE Stars LIKE '%Arnold Schwarzenegger%' ''')
result4 = cursor.fetchall()

# Print the gross collection of movies with the star Arnold Schwarzenegger without repetition
print("Gross collection of movies with the star Arnold Schwarzenegger without repetition:")
for row in result4:
    print("Movie Name:", row[0])
    print("Gross Collection:", row[1])
    print()

# Close the connection
conn.close()


Gross collection of movies with the star Arnold Schwarzenegger without repetition:
Movie Name: Terminator 2: Judgment Day
Gross Collection: 204843350.0

Movie Name: The Terminator
Gross Collection: 38400000.0

Movie Name: Predator
Gross Collection: 59735548.0

Movie Name: Total Recall
Gross Collection: 119394840.0

Movie Name: True Lies
Gross Collection: 146282411.0

Movie Name: Conan the Barbarian
Gross Collection: 39565475.0

Movie Name: Commando
Gross Collection: 35100000.0

Movie Name: Escape Plan
Gross Collection: 25132228.0

Movie Name: The Running Man
Gross Collection: 38122105.0

Movie Name: Last Action Hero
Gross Collection: 50016394.0



In [70]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# 5) Display all details of movies with comedy and action genres without repetition
cursor.execute('''SELECT DISTINCT *
                  FROM second_table
                  WHERE Genre LIKE '%comedy%' AND Genre LIKE '%action%' ''')
result5 = cursor.fetchall()

# Print the details of movies with comedy and action genres without repetition
print("Details of movies with comedy and action genres without repetition:")
for row in result5:
    print(row)
    print()

# Close the connection
conn.close()


Details of movies with comedy and action genres without repetition:
('Gangs of Wasseypur', 'Manoj Bajpayee, Nawazuddin Siddiqui, Tigmanshu Dhulia, Richa Chadha', 99855, 'Action, Comedy, Crime', 99855.0, 'A')

('Sherlock Jr.', 'Buster Keaton, Kathryn McGuire, Joe Keaton, Erwin Connelly', 53068, 'Action, Comedy, Romance', 977375.0, None)

('Guardians of the Galaxy Vol. 3', 'Chris Pratt, Chukwudi Iwuji, Bradley Cooper, Pom Klementieff', 212902, 'Action, Adventure, Comedy', 212902.0, 'UA')

('Lock, Stock and Two Smoking Barrels', 'Jason Flemyng, Dexter Fletcher, Nick Moran, Jason Statham', 597766, 'Action, Comedy, Crime', 3897569.0, 'A')

('Bajrangi Bhaijaan', 'Mehmet Tutkun, Salman Khan, Harshaali Malhotra, Nawazuddin Siddiqui', 92617, 'Action, Adventure, Comedy', 8178001.0, 'UA')

('The General', 'Buster Keaton, Buster Keaton, Marion Mack, Glen Cavender', 94505, 'Action, Adventure, Comedy', 1033895.0, None)

('Hera Pheri', 'Akshay Kumar, Suniel Shetty, Paresh Rawal, Tabu', 69839, 'Action

In [72]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Display all details from both tables where movie names are the same without repetition
cursor.execute('''SELECT DISTINCT *
                  FROM first_table
                  INNER JOIN second_table ON first_table.Movie_Name = second_table.Movie_Name''')
result = cursor.fetchall()

# Print the details where movie names are the same without repetition
print("Details from both tables where movie names are the same without repetition:")
for row in result:
    print(row)
    print()

# Close the connection
conn.close()



Details from both tables where movie names are the same without repetition:
(1, 'The Dark Knight', 'Christopher Nolan, Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine', 152.0, 2008.0, 9.0, 84.0, 'The Dark Knight', 'Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine', 2738965, 'Action, Crime, Drama', 534858444.0, 'UA')

(2, 'The Lord of the Rings: The Return of the King', 'Peter Jackson, Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom', 201.0, 2003.0, 9.0, 94.0, 'The Lord of the Rings: The Return of the King', 'Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom', 1897977, 'Action, Adventure, Drama', 377845905.0, 'U')

(3, 'Spider-Man: Across the Spider-Verse', 'Joaquim Dos Santos, Kemp Powers, Justin K. Thompson, Shameik Moore, Hailee Steinfeld, Brian Tyree Henry, Luna Lauren Velez', 140.0, 2023.0, 8.9, 86.0, 'Spider-Man: Across the Spider-Verse', 'Kemp Powers, Justin K. Thompson, Shameik Moore, Hailee Steinfeld', 182105, 'Animation, Action, Adventure

In [73]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Display all movie names, directors, ratings, and gross collection where the genre is action without repetition
cursor.execute('''SELECT DISTINCT first_table.Movie_Name, first_table.Director_Name, first_table.Ratings, second_table.Gross_Collection
                  FROM first_table
                  INNER JOIN second_table ON first_table.Movie_Name = second_table.Movie_Name
                  WHERE second_table.Genre LIKE '%action%' ''')
result = cursor.fetchall()

# Print the details where the genre is action without repetition
print("Movie names, directors, ratings, and gross collection where the genre is action without repetition:")
for row in result:
    print("Movie Name:", row[0])
    print("Director:", row[1])
    print("Ratings:", row[2])
    print("Gross Collection:", row[3])
    print()

# Close the connection
conn.close()


Movie names, directors, ratings, and gross collection where the genre is action without repetition:
Movie Name: The Dark Knight
Director: Christopher Nolan, Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine
Ratings: 9.0
Gross Collection: 534858444.0

Movie Name: The Lord of the Rings: The Return of the King
Director: Peter Jackson, Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom
Ratings: 9.0
Gross Collection: 377845905.0

Movie Name: Spider-Man: Across the Spider-Verse
Director: Joaquim Dos Santos, Kemp Powers, Justin K. Thompson, Shameik Moore, Hailee Steinfeld, Brian Tyree Henry, Luna Lauren Velez
Ratings: 8.9
Gross Collection: 182105.0

Movie Name: Inception
Director: Christopher Nolan, Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page, Ken Watanabe
Ratings: 8.8
Gross Collection: 292576195.0

Movie Name: The Lord of the Rings: The Fellowship of the Ring
Director: Peter Jackson, Elijah Wood, Ian McKellen, Orlando Bloom, Sean Bean
Ratings: 8.8
Gross Collection: 3

In [74]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Display all details from both tables with the highest gross collection without repetition
cursor.execute('''SELECT DISTINCT *
                  FROM first_table
                  INNER JOIN second_table ON first_table.Movie_Name = second_table.Movie_Name
                  WHERE second_table.Gross_Collection = (SELECT MAX(Gross_Collection) FROM second_table)''')
result = cursor.fetchall()

# Print the details with the highest gross collection without repetition
print("Details from both tables with the highest gross collection without repetition:")
for row in result:
    print(row)
    print()

# Close the connection
conn.close()


Details from both tables with the highest gross collection without repetition:
(136, 'Star Wars: Episode VII - The Force Awakens', 'J.J. Abrams, Daisy Ridley, John Boyega, Oscar Isaac, Domhnall Gleeson', 138.0, 2015.0, 7.8, 80.0, 'Star Wars: Episode VII - The Force Awakens', 'Daisy Ridley, John Boyega, Oscar Isaac, Domhnall Gleeson', 950925, 'Action, Adventure, Sci-Fi', 936662225.0, 'U')



In [75]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Display all details from both tables with the highest ratings without repetition
cursor.execute('''SELECT DISTINCT *
                  FROM first_table
                  INNER JOIN second_table ON first_table.Movie_Name = second_table.Movie_Name
                  WHERE first_table.Ratings = (SELECT MAX(Ratings) FROM first_table)''')
result = cursor.fetchall()

# Print the details with the highest ratings without repetition
print("Details from both tables with the highest ratings without repetition:")
for row in result:
    print(row)
    print()

# Close the connection
conn.close()


Details from both tables with the highest ratings without repetition:
(1, 'The Dark Knight', 'Christopher Nolan, Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine', 152.0, 2008.0, 9.0, 84.0, 'The Dark Knight', 'Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine', 2738965, 'Action, Crime, Drama', 534858444.0, 'UA')

(2, 'The Lord of the Rings: The Return of the King', 'Peter Jackson, Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom', 201.0, 2003.0, 9.0, 94.0, 'The Lord of the Rings: The Return of the King', 'Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom', 1897977, 'Action, Adventure, Drama', 377845905.0, 'U')



# QUERY USING PANDAS SQL

In [95]:


# Connect to SQLite database
conn = sqlite3.connect('movies.db')

# Query to select movies created by directors Christopher and Matt Reeves using LIKE without repetition
query = '''SELECT DISTINCT *
           FROM first_table
           WHERE Director_Name LIKE '%Christopher%' OR Director_Name LIKE '%Matt%' '''

# Execute the query and fetch the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Print the details of movies created by Christopher and Matt Reeves without repetition
print("Details of movies created by Christopher and Matt Reeves without repetition:")
print(df)

# Close the connection
conn.close()



Details of movies created by Christopher and Matt Reeves without repetition:
    Sno                                     Movie_Name  \
0     1                                The Dark Knight   
1     4                                      Inception   
2    22                          The Dark Knight Rises   
3    40                                  Batman Begins   
4    57  Mission: Impossible - Dead Reckoning Part One   
..  ...                                            ...   
72  885                                      Self/less   
73  928                                Army of Thieves   
74  970                                   The Prophecy   
75  973                              Last Man Standing   
76  981                                    Black Death   

                                        Director_Name  Duration    Year  \
0   Christopher Nolan, Christian Bale, Heath Ledge...     152.0  2008.0   
1   Christopher Nolan, Leonardo DiCaprio, Joseph G...     148.0  2010.0   
2

In [96]:
df

,Sno,Movie_Name,Director_Name,Duration,Year,Ratings,Metascore
0,1,The Dark Knight,"Christopher Nolan, Christian Bale, Heath Ledge...",152.0,2008.0,9.0,84.0
1,4,Inception,"Christopher Nolan, Leonardo DiCaprio, Joseph G...",148.0,2010.0,8.8,74.0
2,22,The Dark Knight Rises,"Christopher Nolan, Christian Bale, Tom Hardy, ...",164.0,2012.0,8.4,78.0
3,40,Batman Begins,"Christopher Nolan, Christian Bale, Michael Cai...",140.0,2005.0,8.2,70.0
4,57,Mission: Impossible - Dead Reckoning Part One,"Christopher McQuarrie, Tom Cruise, Hayley Atwe...",163.0,2023.0,8.1,81.0
...,...,...,...,...,...,...,...
72,885,Self/less,"Tarsem Singh, Ryan Reynolds, Natalie Martinez,...",117.0,2015.0,6.5,34.0
73,928,Army of Thieves,"Matthias Schweighöfer, Matthias Schweighöfer, ...",127.0,2021.0,6.4,49.0
74,970,The Prophecy,"Gregory Widen, Christopher Walken, Elias Kotea...",98.0,1995.0,6.4,44.0
75,973,Last Man Standing,"Walter Hill, Bruce Willis, Bruce Dern, William...",101.0,1996.0,6.4,44.0


In [97]:


# Connect to SQLite database
conn = sqlite3.connect('movies.db')

# Query to select movies with duration between 140 and 190 minutes without repetition
query = '''SELECT DISTINCT *
           FROM first_table
           WHERE Duration BETWEEN 140 AND 190'''

# Execute the query and fetch the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Print the details of movies with duration between 140 and 190 minutes without repetition
print("Details of movies with duration between 140 and 190 minutes without repetition:")
print(df)

# Close the connection
conn.close()


Details of movies with duration between 140 and 190 minutes without repetition:
     Sno                                         Movie_Name  \
0      1                                    The Dark Knight   
1      3                Spider-Man: Across the Spider-Verse   
2      4                                          Inception   
3      5  The Lord of the Rings: The Fellowship of the Ring   
4      6              The Lord of the Rings: The Two Towers   
..   ...                                                ...   
154  915                                                War   
155  917                                            Dhoom:2   
156  920                                         Fetih 1453   
157  921                 Batman v Superman: Dawn of Justice   
158  951                                    The Lone Ranger   

                                         Director_Name  Duration    Year  \
0    Christopher Nolan, Christian Bale, Heath Ledge...     152.0  2008.0   
1    Joaqui

In [98]:
df

,Sno,Movie_Name,Director_Name,Duration,Year,Ratings,Metascore
0,1,The Dark Knight,"Christopher Nolan, Christian Bale, Heath Ledge...",152.0,2008.0,9.0,84.0
1,3,Spider-Man: Across the Spider-Verse,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...",140.0,2023.0,8.9,86.0
2,4,Inception,"Christopher Nolan, Leonardo DiCaprio, Joseph G...",148.0,2010.0,8.8,74.0
3,5,The Lord of the Rings: The Fellowship of the Ring,"Peter Jackson, Elijah Wood, Ian McKellen, Orla...",178.0,2001.0,8.8,92.0
4,6,The Lord of the Rings: The Two Towers,"Peter Jackson, Elijah Wood, Ian McKellen, Vigg...",179.0,2002.0,8.8,87.0
...,...,...,...,...,...,...,...
154,915,War,"Siddharth Anand, Hrithik Roshan, Tiger Shroff,...",154.0,2019.0,6.5,NaN
155,917,Dhoom:2,"Sanjay Gadhvi, Hrithik Roshan, Abhishek Bachch...",152.0,2006.0,6.5,NaN
156,920,Fetih 1453,"Faruk Aksoy, Devrim Evin, Ibrahim Celikkol, Di...",162.0,2012.0,6.5,NaN
157,921,Batman v Superman: Dawn of Justice,"Zack Snyder, Ben Affleck, Henry Cavill, Amy Ad...",151.0,2016.0,6.4,44.0


In [99]:


# Connect to SQLite database
conn = sqlite3.connect('movies.db')

# Query to select movies with ratings above 7 in ascending order without repetition
query = '''SELECT DISTINCT *
           FROM first_table
           WHERE Ratings > 7
           ORDER BY Ratings ASC'''

# Execute the query and fetch the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Print the details of movies with ratings above 7 in ascending order without repetition
print("Details of movies with ratings above 7 in ascending order without repetition:")
print(df)

# Close the connection
conn.close()


Details of movies with ratings above 7 in ascending order without repetition:
     Sno                                         Movie_Name  \
0    432                                          The Flash   
1    433                                       Extraction 2   
2    434                                Mission: Impossible   
3    435                                       Man of Steel   
4    436                                          The Mummy   
..   ...                                                ...   
489    5  The Lord of the Rings: The Fellowship of the Ring   
490    6              The Lord of the Rings: The Two Towers   
491    3                Spider-Man: Across the Spider-Verse   
492    1                                    The Dark Knight   
493    2      The Lord of the Rings: The Return of the King   

                                         Director_Name  Duration    Year  \
0    Andy Muschietti, Ezra Miller, Michael Keaton, ...     144.0  2023.0   
1    Sam Harg

In [100]:
df

,Sno,Movie_Name,Director_Name,Duration,Year,Ratings,Metascore
0,432,The Flash,"Andy Muschietti, Ezra Miller, Michael Keaton, ...",144.0,2023.0,7.1,56.0
1,433,Extraction 2,"Sam Hargrave, Chris Hemsworth, Golshifteh Fara...",122.0,2023.0,7.1,57.0
2,434,Mission: Impossible,"Brian De Palma, Tom Cruise, Jon Voight, Emmanu...",110.0,1996.0,7.1,59.0
3,435,Man of Steel,"Zack Snyder, Henry Cavill, Amy Adams, Michael ...",143.0,2013.0,7.1,55.0
4,436,The Mummy,"Stephen Sommers, Brendan Fraser, Rachel Weisz,...",124.0,1999.0,7.1,48.0
...,...,...,...,...,...,...,...
489,5,The Lord of the Rings: The Fellowship of the Ring,"Peter Jackson, Elijah Wood, Ian McKellen, Orla...",178.0,2001.0,8.8,92.0
490,6,The Lord of the Rings: The Two Towers,"Peter Jackson, Elijah Wood, Ian McKellen, Vigg...",179.0,2002.0,8.8,87.0
491,3,Spider-Man: Across the Spider-Verse,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...",140.0,2023.0,8.9,86.0
492,1,The Dark Knight,"Christopher Nolan, Christian Bale, Heath Ledge...",152.0,2008.0,9.0,84.0


In [101]:

# Connect to SQLite database
conn = sqlite3.connect('movies.db')

# Query to select all movie names in descending order without repetition
query = '''SELECT DISTINCT Movie_Name
           FROM first_table
           ORDER BY Movie_Name DESC'''

# Execute the query and fetch the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Print the movie names in descending order without repetition
print("Movie names in descending order without repetition:")
print(df)

# Close the connection
conn.close()


Movie names in descending order without repetition:
                       Movie_Name
0                    Ölümlü Dünya
1          Zombieland: Double Tap
2                      Zombieland
3                        Zatôichi
4    Zack Snyder's Justice League
..                            ...
990                     16 Blocks
991                      13 Hours
992                     12 Strong
993               1 - Nenokkadine
994                           '71

[995 rows x 1 columns]


In [102]:
df

,Movie_Name
0,Ölümlü Dünya
1,Zombieland: Double Tap
2,Zombieland
3,Zatôichi
4,Zack Snyder's Justice League
...,...
990,16 Blocks
991,13 Hours
992,12 Strong
993,1 - Nenokkadine


In [103]:


# Connect to SQLite database
conn = sqlite3.connect('movies.db')

# Query to select movie names that start with 'P' and their rating is greater than 7 without repetition
query = '''SELECT DISTINCT Movie_Name
           FROM first_table
           WHERE Movie_Name LIKE 'P%' AND Ratings > 7'''

# Execute the query and fetch the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Print the movie names that start with 'P' and their rating is greater than 7 without repetition
print("Movie names that start with 'P' and their rating is greater than 7 without repetition:")
print(df)

# Close the connection
conn.close()


Movie names that start with 'P' and their rating is greater than 7 without repetition:
                                           Movie_Name
0                                    Paan Singh Tomar
1   Pirates of the Caribbean: The Curse of the Bla...
2                             Per un pugno di dollari
3                                            Predator
4                                  Ponniyin Selvan: I
5                           Pushpa: The Rise - Part 1
6                                      Predestination
7          Pirates of the Caribbean: Dead Man's Chest
8                                        Patriots Day
9                                         Point Break
10                                               Prey
11           Pirates of the Caribbean: At World's End
12                                      Planet Terror
13                                            Payback


In [104]:
df

,Movie_Name
0,Paan Singh Tomar
1,Pirates of the Caribbean: The Curse of the Bla...
2,Per un pugno di dollari
3,Predator
4,Ponniyin Selvan: I
5,Pushpa: The Rise - Part 1
6,Predestination
7,Pirates of the Caribbean: Dead Man's Chest
8,Patriots Day
9,Point Break


In [105]:


# Connect to SQLite database
conn = sqlite3.connect('movies.db')

# Query to select distinct movie names with star Arnold Schwarzenegger in ascending order
query = '''SELECT DISTINCT Movie_Name
           FROM second_table
           WHERE Stars LIKE '%Arnold Schwarzenegger%'
           ORDER BY Movie_Name ASC'''

# Execute the query and fetch the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Print the movie names with star Arnold Schwarzenegger in ascending order without repetition
print("Movie names with star Arnold Schwarzenegger in ascending order without repetition:")
print(df)

# Close the connection
conn.close()


Movie names with star Arnold Schwarzenegger in ascending order without repetition:
                   Movie_Name
0                    Commando
1         Conan the Barbarian
2                 Escape Plan
3            Last Action Hero
4                    Predator
5  Terminator 2: Judgment Day
6             The Running Man
7              The Terminator
8                Total Recall
9                   True Lies


In [106]:
df

,Movie_Name
0,Commando
1,Conan the Barbarian
2,Escape Plan
3,Last Action Hero
4,Predator
5,Terminator 2: Judgment Day
6,The Running Man
7,The Terminator
8,Total Recall
9,True Lies


In [107]:

# Connect to SQLite database
conn = sqlite3.connect('movies.db')

# Query to select all details of the movie with the highest number of votes without repetition
query = '''SELECT *
           FROM second_table
           WHERE Votes = (SELECT MAX(Votes) FROM second_table)
           LIMIT 1'''

# Execute the query and fetch the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Print the details of the movie with the highest number of votes without repetition
print("Details of the movie with the highest number of votes without repetition:")
print(df)

# Close the connection
conn.close()


Details of the movie with the highest number of votes without repetition:
        Movie_Name                                              Stars  \
0  The Dark Knight  Christian Bale, Heath Ledger, Aaron Eckhart, M...   

     Votes                 Genre  Gross_Collection Certification  
0  2738965  Action, Crime, Drama       534858444.0            UA  


In [108]:
df

,Movie_Name,Stars,Votes,Genre,Gross_Collection,Certification
0,The Dark Knight,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",2738965,"Action, Crime, Drama",534858444.0,UA


In [117]:


# Connect to SQLite database
conn = sqlite3.connect('movies.db')

# Query to select movie names with gross collections in descending order without repetition
query = '''SELECT DISTINCT Movie_Name, Gross_Collection
           FROM second_table
           ORDER BY Gross_Collection DESC'''

# Execute the query and fetch the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Print the movie names with gross collections in descending order without repetition
print("Movie names with gross collections in descending order without repetition:")
print(df)

# Close the connection
conn.close()


Movie names with gross collections in descending order without repetition:
                                     Movie_Name  Gross_Collection
0    Star Wars: Episode VII - The Force Awakens       936662225.0
1                             Avengers: Endgame       858373000.0
2                       Spider-Man: No Way Home       804747988.0
3                                        Avatar       760507625.0
4                             Top Gun: Maverick       718732821.0
..                                          ...               ...
995                              Tropa de Elite            8060.0
996                                     Ajeossi            6460.0
997                               The Salvation            5000.0
998                   Knockin' on Heaven's Door            3296.0
999                                Son of a Gun            1411.0

[1000 rows x 2 columns]


In [118]:
df

,Movie_Name,Gross_Collection
0,Star Wars: Episode VII - The Force Awakens,936662225.0
1,Avengers: Endgame,858373000.0
2,Spider-Man: No Way Home,804747988.0
3,Avatar,760507625.0
4,Top Gun: Maverick,718732821.0
...,...,...
995,Tropa de Elite,8060.0
996,Ajeossi,6460.0
997,The Salvation,5000.0
998,Knockin' on Heaven's Door,3296.0


In [115]:


# Connect to SQLite database
conn = sqlite3.connect('movies.db')

# Query to select the gross collection of movies with the star Arnold without repetition
query = '''SELECT DISTINCT Movie_Name, Gross_Collection
           FROM second_table
           WHERE Stars LIKE '%Arnold%'
           ORDER BY Gross_Collection'''

# Execute the query and fetch the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Print the gross collection of movies with the star Arnold without repetition
print("Gross collection of movies with the star Arnold without repetition:")
print(df)

# Close the connection
conn.close()


Gross collection of movies with the star Arnold without repetition:
                    Movie_Name  Gross_Collection
0                  Escape Plan        25132228.0
1                     Commando        35100000.0
2              The Running Man        38122105.0
3               The Terminator        38400000.0
4          Conan the Barbarian        39565475.0
5             Last Action Hero        50016394.0
6                     Predator        59735548.0
7                 Total Recall       119394840.0
8                    True Lies       146282411.0
9                    The Mummy       155247825.0
10           The Mummy Returns       202019785.0
11  Terminator 2: Judgment Day       204843350.0


In [116]:
df

,Movie_Name,Gross_Collection
0,Escape Plan,25132228.0
1,Commando,35100000.0
2,The Running Man,38122105.0
3,The Terminator,38400000.0
4,Conan the Barbarian,39565475.0
5,Last Action Hero,50016394.0
6,Predator,59735548.0
7,Total Recall,119394840.0
8,True Lies,146282411.0
9,The Mummy,155247825.0


In [120]:

# Connect to SQLite database
conn = sqlite3.connect('movies.db')

# Query to select all details of movies with comedy and action genres without repetition
query = '''SELECT DISTINCT *
           FROM second_table
           WHERE Genre LIKE '%comedy%' AND Genre LIKE '%action%' '''

# Execute the query and fetch the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Print the details of movies with comedy and action genres without repetition
print("Details of movies with comedy and action genres without repetition:")
print(df)

# Close the connection
conn.close()


Details of movies with comedy and action genres without repetition:
                              Movie_Name  \
0                     Gangs of Wasseypur   
1                           Sherlock Jr.   
2         Guardians of the Galaxy Vol. 3   
3    Lock, Stock and Two Smoking Barrels   
4                      Bajrangi Bhaijaan   
..                                   ...   
160                  Here Comes the Boom   
161                         Tango & Cash   
162                        Let's Be Cops   
163                           Redirected   
164    Operation Fortune: Ruse de Guerre   

                                                 Stars   Votes  \
0    Manoj Bajpayee, Nawazuddin Siddiqui, Tigmanshu...   99855   
1    Buster Keaton, Kathryn McGuire, Joe Keaton, Er...   53068   
2    Chris Pratt, Chukwudi Iwuji, Bradley Cooper, P...  212902   
3    Jason Flemyng, Dexter Fletcher, Nick Moran, Ja...  597766   
4    Mehmet Tutkun, Salman Khan, Harshaali Malhotra...   92617   
..     

In [121]:
df

,Movie_Name,Stars,Votes,Genre,Gross_Collection,Certification
0,Gangs of Wasseypur,"Manoj Bajpayee, Nawazuddin Siddiqui, Tigmanshu...",99855,"Action, Comedy, Crime",99855.0,A
1,Sherlock Jr.,"Buster Keaton, Kathryn McGuire, Joe Keaton, Er...",53068,"Action, Comedy, Romance",977375.0,None
2,Guardians of the Galaxy Vol. 3,"Chris Pratt, Chukwudi Iwuji, Bradley Cooper, P...",212902,"Action, Adventure, Comedy",212902.0,UA
3,"Lock, Stock and Two Smoking Barrels","Jason Flemyng, Dexter Fletcher, Nick Moran, Ja...",597766,"Action, Comedy, Crime",3897569.0,A
4,Bajrangi Bhaijaan,"Mehmet Tutkun, Salman Khan, Harshaali Malhotra...",92617,"Action, Adventure, Comedy",8178001.0,UA
...,...,...,...,...,...,...
160,Here Comes the Boom,"Kevin James, Salma Hayek, Henry Winkler, Greg ...",93708,"Action, Comedy, Sport",45290318.0,PG
161,Tango & Cash,"Albert Magnoli, Sylvester Stallone, Kurt Russe...",110003,"Action, Comedy, Crime",63408614.0,UA
162,Let's Be Cops,"Jake Johnson, Damon Wayans Jr., Rob Riggle, Ni...",137557,"Action, Comedy, Crime",82390774.0,R
163,Redirected,"Vinnie Jones, Scot Williams, Gil Darnell, Oliv...",34350,"Action, Comedy, Crime",34350.0,None


In [122]:


# Connect to SQLite database
conn = sqlite3.connect('movies.db')

# Query to select all details from both tables where movie names are the same without repetition
query = '''SELECT DISTINCT *
           FROM first_table
           INNER JOIN second_table ON first_table.Movie_Name = second_table.Movie_Name'''

# Execute the query and fetch the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Print the details from both tables where movie names are the same without repetition
print("Details from both tables where movie names are the same without repetition:")
print(df)

# Close the connection
conn.close()


Details from both tables where movie names are the same without repetition:
        Sno                                         Movie_Name  \
0         1                                    The Dark Knight   
1         2      The Lord of the Rings: The Return of the King   
2         3                Spider-Man: Across the Spider-Verse   
3         4                                          Inception   
4         5  The Lord of the Rings: The Fellowship of the Ring   
...     ...                                                ...   
1005    996                           Baiohazâdo: Dijenerêshon   
1006    997                                      Dnevnoy dozor   
1007    998                                         Redirected   
1008    999                   Transformers: Rise of the Beasts   
1009  1,000                  Operation Fortune: Ruse de Guerre   

                                          Director_Name  Duration    Year  \
0     Christopher Nolan, Christian Bale, Heath Ledge..

In [123]:
df

,Sno,Movie_Name,Director_Name,Duration,Year,Ratings,Metascore,Movie_Name,Stars,Votes,Genre,Gross_Collection,Certification
0,1,The Dark Knight,"Christopher Nolan, Christian Bale, Heath Ledge...",152.0,2008.0,9.0,84.0,The Dark Knight,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",2738965,"Action, Crime, Drama",534858444.0,UA
1,2,The Lord of the Rings: The Return of the King,"Peter Jackson, Elijah Wood, Viggo Mortensen, I...",201.0,2003.0,9.0,94.0,The Lord of the Rings: The Return of the King,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",1897977,"Action, Adventure, Drama",377845905.0,U
2,3,Spider-Man: Across the Spider-Verse,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...",140.0,2023.0,8.9,86.0,Spider-Man: Across the Spider-Verse,"Kemp Powers, Justin K. Thompson, Shameik Moore...",182105,"Animation, Action, Adventure",182105.0,U
3,4,Inception,"Christopher Nolan, Leonardo DiCaprio, Joseph G...",148.0,2010.0,8.8,74.0,Inception,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",2430566,"Action, Adventure, Sci-Fi",292576195.0,UA
4,5,The Lord of the Rings: The Fellowship of the Ring,"Peter Jackson, Elijah Wood, Ian McKellen, Orla...",178.0,2001.0,8.8,92.0,The Lord of the Rings: The Fellowship of the Ring,"Elijah Wood, Ian McKellen, Orlando Bloom, Sean...",1926413,"Action, Adventure, Drama",315544750.0,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,996,Baiohazâdo: Dijenerêshon,"Makoto Kamiya, Paul Mercier, Alyson Court, Lau...",97.0,2008.0,6.4,NaN,Baiohazâdo: Dijenerêshon,"Paul Mercier, Alyson Court, Laura Bailey, Roge...",32289,"Animation, Action, Horror",32289.0,R
1006,997,Dnevnoy dozor,"Timur Bekmambetov, Konstantin Khabenskiy, Mari...",132.0,2006.0,6.4,59.0,Dnevnoy dozor,"Konstantin Khabenskiy, Mariya Poroshina, Vladi...",34964,"Action, Fantasy, Horror",450686.0,R
1007,998,Redirected,"Emilis Velyvis, Vinnie Jones, Scot Williams, G...",99.0,2014.0,6.4,NaN,Redirected,"Vinnie Jones, Scot Williams, Gil Darnell, Oliv...",34350,"Action, Comedy, Crime",34350.0,None
1008,999,Transformers: Rise of the Beasts,"Steven Caple Jr., Anthony Ramos, Dominique Fis...",127.0,2023.0,6.3,42.0,Transformers: Rise of the Beasts,"Anthony Ramos, Dominique Fishback, Luna Lauren...",44774,"Action, Adventure, Sci-Fi",44774.0,UA


In [128]:
import pandas as pd
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('movies.db')

# Query to select movie names, directors, ratings, and gross collections where the genre is action without repetition
query = '''SELECT DISTINCT second_table.Movie_Name, second_table.Stars AS Director, second_table.Votes, second_table.Gross_Collection
           FROM second_table
           WHERE second_table.Genre LIKE '%action%' '''

# Execute the query and fetch the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Print the movie names, directors, ratings, and gross collections where the genre is action without repetition
print("Movie names, directors, votes, and gross collections where the genre is action without repetition:")
print(df)

# Close the connection
conn.close()



Movie names, directors, votes, and gross collections where the genre is action without repetition:
                                            Movie_Name  \
0                                      The Dark Knight   
1        The Lord of the Rings: The Return of the King   
2                  Spider-Man: Across the Spider-Verse   
3                                            Inception   
4    The Lord of the Rings: The Fellowship of the Ring   
..                                                 ...   
995                           Baiohazâdo: Dijenerêshon   
996                                      Dnevnoy dozor   
997                                         Redirected   
998                   Transformers: Rise of the Beasts   
999                  Operation Fortune: Ruse de Guerre   

                                              Director    Votes  \
0    Christian Bale, Heath Ledger, Aaron Eckhart, M...  2738965   
1    Elijah Wood, Viggo Mortensen, Ian McKellen, Or...  1897977   
2  

In [129]:
df

,Movie_Name,Director,Votes,Gross_Collection
0,The Dark Knight,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",2738965,534858444.0
1,The Lord of the Rings: The Return of the King,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",1897977,377845905.0
2,Spider-Man: Across the Spider-Verse,"Kemp Powers, Justin K. Thompson, Shameik Moore...",182105,182105.0
3,Inception,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",2430566,292576195.0
4,The Lord of the Rings: The Fellowship of the Ring,"Elijah Wood, Ian McKellen, Orlando Bloom, Sean...",1926413,315544750.0
...,...,...,...,...
995,Baiohazâdo: Dijenerêshon,"Paul Mercier, Alyson Court, Laura Bailey, Roge...",32289,32289.0
996,Dnevnoy dozor,"Konstantin Khabenskiy, Mariya Poroshina, Vladi...",34964,450686.0
997,Redirected,"Vinnie Jones, Scot Williams, Gil Darnell, Oliv...",34350,34350.0
998,Transformers: Rise of the Beasts,"Anthony Ramos, Dominique Fishback, Luna Lauren...",44774,44774.0


In [133]:
import pandas as pd
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('movies.db')

# Query to select all details from both tables with the highest gross collection without repetition
query = '''SELECT DISTINCT *
           FROM first_table
           INNER JOIN second_table ON first_table.Movie_Name = second_table.Movie_Name
           WHERE second_table.Gross_Collection = (SELECT MAX(Gross_Collection) FROM second_table)'''

# Execute the query and fetch the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Print all details from both tables with the highest gross collection without repetition
print("Details from both tables with the highest gross collection without repetition:")
print(df)

# Close the connection
conn.close()


Details from both tables with the highest gross collection without repetition:
   Sno                                  Movie_Name  \
0  136  Star Wars: Episode VII - The Force Awakens   

                                       Director_Name  Duration    Year  \
0  J.J. Abrams, Daisy Ridley, John Boyega, Oscar ...     138.0  2015.0   

   Ratings  Metascore                                  Movie_Name  \
0      7.8       80.0  Star Wars: Episode VII - The Force Awakens   

                                               Stars   Votes  \
0  Daisy Ridley, John Boyega, Oscar Isaac, Domhna...  950925   

                       Genre  Gross_Collection Certification  
0  Action, Adventure, Sci-Fi       936662225.0             U  


In [134]:
df

,Sno,Movie_Name,Director_Name,Duration,Year,Ratings,Metascore,Movie_Name,Stars,Votes,Genre,Gross_Collection,Certification
0,136,Star Wars: Episode VII - The Force Awakens,"J.J. Abrams, Daisy Ridley, John Boyega, Oscar ...",138.0,2015.0,7.8,80.0,Star Wars: Episode VII - The Force Awakens,"Daisy Ridley, John Boyega, Oscar Isaac, Domhna...",950925,"Action, Adventure, Sci-Fi",936662225.0,U
